In [1]:
import os

go_up_n_directories = lambda path, n: os.path.abspath(
    os.path.join(*([os.path.dirname(path)] + [".."] * n))
)
os.chdir(go_up_n_directories(os.getcwd(), 3))  # run once (otherwise restart kernel)

In [2]:
# DEPENDENCIES
# Python native
import functools

# Data handling
import ocpa.algo.predictive_monitoring.factory as feature_factory

# PyG
import torch
import torch.nn.functional as F
import torch.optim as O
import torch_geometric.transforms as T

import utilities.torch_utils
from models.definitions.geometric_models import HigherOrderGNN
from utilities import hetero_data_utils, hetero_experiment_utils

# Print system info
utilities.torch_utils.print_system_info()
utilities.torch_utils.print_torch_info()

# INITIAL CONFIGURATION
cs_hoeg_config = {
    "model_output_path": "models/CS/hoeg",
    "STORAGE_PATH": "data/CS/feature_encodings/HOEG/hoeg",
    "SPLIT_FEATURE_STORAGE_FILE": "CS_split_[C2_P2_P3_O3_eas].fs",
    "OBJECTS_DATA_DICT": "cs_ofg+oi_graph+krs_node_map+krv_node_map+cv_node_map.pkl",
    "events_target_label": (feature_factory.EVENT_REMAINING_TIME, ()),
    "objects_target_label": "@@object_lifecycle_duration",
    "graph_level_target": False,
    "regression_task": True,
    "target_node_type": "event",
    "object_types": ["krs", "krv", "cv"],
    "meta_data": (
        ["event", "krs", "krv", "cv"],
        [
            ("event", "follows", "event"),
            ("krs", "interacts", "event"),
            ("krv", "interacts", "event"),
            ("cv", "interacts", "event"),
        ],
    ),
    "BATCH_SIZE": 16,
    "RANDOM_SEED": 42,
    "EPOCHS": 30,
    "early_stopping": 4,
    "optimizer": O.Adam,
    "optimizer_settings": {
        "lr": 0.001,
        "betas": (0.9, 0.999),
        "eps": 1e-08,
        "weight_decay": 0,
        "amsgrad": False,
    },
    "loss_fn": torch.nn.L1Loss(),
    "verbose": True,
    "skip_cache": False,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "squeeze": True,
    "track_time": True,
}

# CONFIGURATION ADAPTATIONS may be set here
# cs_hoeg_config["device"] = torch.device("cpu")
# cs_hoeg_config['skip_cache'] = True

CPU: Intel(R) Core(TM) i5-7500 CPU @ 3.40GHz (4x)
Total CPU memory: 46.93GB
Available CPU memory: 29.55GB
GPU: NVIDIA GeForce GTX 960
Total GPU memory: 4096.0MB
Available GPU memory: 3172.0MB
Platform: Linux-6.2.0-31-generic-x86_64-with-glibc2.35
Torch version: 1.13.1+cu117
Cuda available: True
Torch geometric version: 2.3.1


In [3]:
# DATA PREPARATION
transformations = [
    hetero_data_utils.ToUndirected(
        exclude_edge_types=[("event", "follows", "event")]
    ),  #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    hetero_data_utils.AddObjectSelfLoops(),  # Prepares object-object relations, which are filled when `T.AddSelfLoops()` is executed
    T.AddSelfLoops(),  # Add self-loops to the graph
    T.NormalizeFeatures(),  # Normalize node features of the graph
]
# Get data and dataloaders
ds_train, ds_val, ds_test = hetero_data_utils.load_hetero_datasets(
    storage_path=cs_hoeg_config["STORAGE_PATH"],
    split_feature_storage_file=cs_hoeg_config["SPLIT_FEATURE_STORAGE_FILE"],
    objects_data_file=cs_hoeg_config["OBJECTS_DATA_DICT"],
    event_node_label_key=cs_hoeg_config["events_target_label"],
    object_nodes_label_key=cs_hoeg_config["objects_target_label"],
    edge_types=cs_hoeg_config["meta_data"][1],
    object_node_types=cs_hoeg_config["object_types"],
    graph_level_target=cs_hoeg_config["graph_level_target"],
    transform=T.Compose(transformations),
    train=True,
    val=True,
    test=True,
    skip_cache=cs_hoeg_config["skip_cache"],
    debug=True,
)
for data in ds_val:
    data: hetero_data_utils.HeteroData
    if data.metadata() != cs_hoeg_config["meta_data"]:
        cs_hoeg_config["meta_data"] = data.metadata()
        break
(
    train_loader,
    val_loader,
    test_loader,
) = hetero_data_utils.hetero_dataloaders_from_datasets(
    batch_size=cs_hoeg_config["BATCH_SIZE"],
    ds_train=ds_train,
    ds_val=ds_val,
    ds_test=ds_test,
    num_workers=3,
    seed_worker=functools.partial(
        utilities.torch_utils.seed_worker, state=cs_hoeg_config["RANDOM_SEED"]
    ),
    generator=torch.Generator().manual_seed(cs_hoeg_config["RANDOM_SEED"]),
)

In [5]:
hetero_data_utils.print_hetero_dataset_summaries(ds_train, ds_val, ds_test)

Train set


  0%|          | 0/15638 [00:00<?, ?it/s]

100%|██████████| 15638/15638 [00:18<00:00, 843.70it/s]


HOEG (#graphs=15638):
+------------+----------+----------+
|            |   #nodes |   #edges |
|------------+----------+----------|
| mean       |     25.3 |     70.9 |
| std        |     13.1 |     39.4 |
| min        |      7   |     16   |
| quantile25 |     16   |     43   |
| median     |     23   |     64   |
| quantile75 |     32   |     91   |
| max        |    161   |    478   |
+------------+----------+----------+ 

Validation set
HOEG (#graphs=6255):
+------------+----------+----------+
|            |   #nodes |   #edges |
|------------+----------+----------|
| mean       |     25.4 |     71.1 |
| std        |     13.2 |     39.5 |
| min        |      7   |     16   |
| quantile25 |     16   |     43   |
| median     |     23   |     64   |
| quantile75 |     32   |     91   |
| max        |    146   |    433   |
+------------+----------+----------+ 

Test set
HOEG (#graphs=9384):
+------------+----------+----------+
|            |   #nodes |   #edges |
|------------+------

In [7]:
invalid_train_graphs = hetero_data_utils.validate_cs_hoeg_dataset(
    ds_train, verbose=False
)
invalid_val_graphs = hetero_data_utils.validate_cs_hoeg_dataset(ds_val, verbose=False)
invalid_test_graphs = hetero_data_utils.validate_cs_hoeg_dataset(ds_test, verbose=False)

Event node indices valid in all batches for edge type event-event:  True
KRS node indices valid in all batches for edge type krs-event:  True
KRV node indices valid in all batches for edge type krv-event:  True
CV node indices valid in all batches for edge type cv-event:  True
Event node indices valid in all batches for edge type krs-event:  True
Event node indices valid in all batches for edge type krv-event:  True
Event node indices valid in all batches for edge type cv-event:  True
KRS node indices valid in all batches for edge type krs-krs:  True
KRV node indices valid in all batches for edge type krv-krv:  True
CV node indices valid in all batches for edge type cv-cv:  True
HOEG dataset valid:  True


In [4]:
cs_hoeg_config["verbose"] = True
cs_hoeg_config["squeeze"] = True
cs_hoeg_config["model_output_path"] = "models/CS/hoeg/exp_information_leakage"
cs_hoeg_config["device"] = torch.device("cpu")

# lr_range = [0.01, 0.001]
# hidden_dim_range = [8, 16, 24, 32, 48, 64, 128, 256]
lr_range = [0.001]
hidden_dim_range = [64]
for lr in lr_range:
    for hidden_dim in hidden_dim_range:
        hetero_experiment_utils.run_hoeg_experiment_configuration(
            HigherOrderGNN,
            lr=lr,
            hidden_dim=hidden_dim,
            train_loader=train_loader,
            val_loader=val_loader,
            test_loader=test_loader,
            hoeg_config=cs_hoeg_config,
        )


lr=0.001, hidden_dim=64:
Training started, progress available in Tensorboard
EPOCH 0:


/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/nn/to_hetero_transformer.py:379: UserWarning: 'acts.0' will be duplicated, but its parameters cannot be reset. To suppress this warning, add a 'reset_parameters()' method to 'acts.0'
  warnings.warn(
/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/nn/to_hetero_transformer.py:379: UserWarning: 'acts.1' will be duplicated, but its parameters cannot be reset. To suppress this warning, add a 'reset_parameters()' method to 'acts.1'
  warnings.warn(
100it [00:54,  5.57it/s]

  batch 100 loss: 0.7125392013788223


200it [01:13,  3.54it/s]

  batch 200 loss: 0.7612828561663627


...

  batch 1300 loss: 0.6854468557238579


1369it [01:46, 12.82it/s]


Epoch loss -> train: 0.6854468557238579 valid: 0.6951386332511902
Early stopping after 13 epochs.


100%|██████████| 294/294 [00:19<00:00, 15.43it/s]

lr=0.001, hidden_dim=64:
    73868 parameters
    1:48:19.088563 H:m:s
    0.6951


In [ ]:
x = torch.randint(9, (5, 18))
print(x)
xsize = x.size()
print("xsize:", xsize)
x = x.view(-1, 1)
# print(x)
x.shape

tensor([[5, 0, 4, 4, 0, 1, 5, 1, 2, 0, 0, 6, 7, 5, 2, 2, 6, 0],
        [3, 1, 2, 0, 1, 2, 4, 5, 4, 3, 4, 7, 1, 5, 5, 8, 4, 2],
        [3, 1, 0, 1, 8, 4, 1, 2, 3, 1, 2, 0, 2, 4, 3, 4, 4, 1],
        [8, 8, 5, 4, 8, 1, 6, 4, 7, 5, 7, 1, 2, 3, 0, 2, 2, 5],
        [4, 7, 8, 0, 6, 0, 1, 2, 1, 1, 2, 1, 5, 1, 5, 0, 4, 0]])
xsize: torch.Size([5, 18])


torch.Size([90, 1])

In [55]:
import torch.nn as nn
import torch_geometric.nn as pygnn
from models.definitions.geometric_models import GraphModel

cs_hoeg_config["verbose"] = False
cs_hoeg_config["squeeze"] = False
cs_hoeg_config["pre_forward_view"] = True
cs_hoeg_config["track_time"] = True
cs_hoeg_config["model_output_path"] = "models/CS/hoeg/exp_v2"
cs_hoeg_config["device"] = torch.device("cpu")
# cs_hoeg_config["device"] = torch.device('cuda')


class GNN(GraphModel):
    def __init__(
        self,
        hidden_channels: int = 64,
        out_channels: int = 1,
        pre_forward_view: bool = False,
        squeeze: bool = True,
    ):
        super().__init__()
        self.squeeze = squeeze
        self.conv1 = pygnn.GraphConv(-1, hidden_channels)
        self.act1 = nn.PReLU()
        self.lin_out = pygnn.Linear(-1, out_channels)

    def forward(self, x, edge_index, batch=None):
        x = self.conv1(x, edge_index)
        x = self.act1(x)
        x = self.lin_out(x)
        x = torch.squeeze(x) if self.squeeze else x
        return x


lr_range = [0.01, 0.001]
hidden_dim_range = [8, 16, 24, 32, 48, 64, 128, 256]
for lr in lr_range:
    for hidden_dim in hidden_dim_range:
        hetero_experiment_utils.run_hoeg_experiment_configuration(
            GNN,
            lr=lr,
            hidden_dim=hidden_dim,
            train_loader=train_loader,
            val_loader=val_loader,
            test_loader=test_loader,
            hoeg_config=cs_hoeg_config,
        )


lr=0.01, hidden_dim=8:


/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/nn/to_hetero_transformer.py:379: UserWarning: 'act1' will be duplicated, but its parameters cannot be reset. To suppress this warning, add a 'reset_parameters()' method to 'act1'
  warnings.warn(
/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([443])) that is different to the input size (torch.Size([443, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([422])) that is different to the input size (torch.Size([422, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.redu

IndexError: Encountered an index error. Please ensure that all indices in 'edge_index' point to valid indices in the interval [0, 15] (got interval [0, 25])

In [ ]:
def evaluate_hetero_model(
    target_node_type: str,
    model: GraphModel,
    dataloader: DataLoader,
    metric: Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
    device: torch.device = torch.device("cpu"),
    verbose: bool = False,
    squeeze_required: bool = True,
) -> torch.Tensor:
    with torch.no_grad():

        def _eval_batch(batch, model):
            batch_inputs, batch_adjacency_matrix, batch_labels = (
                batch.x_dict,
                batch.edge_index_dict,
                batch[target_node_type].y,
            )
            return (
                model(
                    batch_inputs,
                    edge_index=batch_adjacency_matrix
                    # , batch=batch[target_node_type].batch,
                ),
                batch_labels,
            )

        model.eval()
        model.train(False)
        model.to(device)
        y_preds = torch.tensor([]).to(device)
        y_true = torch.tensor([]).to(device)
        for batch in tqdm(dataloader, disable=not (verbose)):
            batch.to(device)
            batch_y_preds, batch_y_true = _eval_batch(batch, model)
            # append
            y_preds = torch.cat(
                (y_preds, batch_y_preds[target_node_type].view(-1, 25).mean(dim=1))
            )
            y_true = torch.cat((y_true, batch_y_true))
        if squeeze_required:
            y_preds = torch.squeeze(y_preds)
    return metric(y_preds.to(device), y_true.to(device))


def evaluate_best_model(
    target_node_type: str,
    model_state_dict_path: str,
    model: GraphModel,
    metric: Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
    device: torch.device,
    train_loader: Union[DataLoader, None] = None,
    val_loader: Union[DataLoader, None] = None,
    test_loader: Union[DataLoader, None] = None,
    verbose: bool = True,
    squeeze_required: bool = True,
) -> dict[str, torch.Tensor]:
    best_state_dict = torch.load(model_state_dict_path, map_location=device)

    model.load_state_dict(best_state_dict)
    model.eval()
    kwargs = {
        "target_node_type": target_node_type,
        "model": model,
        "metric": metric,
        "device": device,
        "verbose": verbose,
        "squeeze_required": squeeze_required,
    }
    evaluation = {}
    if train_loader:
        evaluation |= {
            f"Train {metric}": evaluate_hetero_model(dataloader=train_loader, **kwargs)
        }
    if val_loader:
        evaluation |= {
            f"Val {metric}": evaluate_hetero_model(dataloader=val_loader, **kwargs)
        }
    if test_loader:
        evaluation |= {
            f"Test {metric}": evaluate_hetero_model(dataloader=test_loader, **kwargs)
        }
    return evaluation

In [ ]:
cs_hoeg_config["track_time"] = True
cs_hoeg_config["verbose"] = True
cs_hoeg_config["squeeze"] = True
cs_hoeg_config["EPOCHS"] = 1
# cs_hoeg_config["device"] = torch.device("cpu")

lr_range = [0.01, 0.001]
hidden_dim_range = [8, 16, 24, 32, 48, 64, 128, 256]
for lr in lr_range:
    for hidden_dim in hidden_dim_range:
        hetero_experiment_utils.run_hoeg_experiment_configuration(
            HeteroHigherOrderGNN,
            lr=lr,
            hidden_dim=hidden_dim,
            train_loader=train_loader,
            val_loader=val_loader,
            test_loader=test_loader,
            hoeg_config=cs_hoeg_config,
        )

## legacy

In [ ]:
# MODEL INITIATION
# TODO: try custom Heterogeneous GNN Architecture (without to_hetero())
class HeteroHigherOrderGNN(GraphModel):
    def __init__(
        self,
        hidden_channels: int = 8,
        out_channels: int = 1,
        regression_target: bool = True,
    ):
        super().__init__()
        self.conv1 = pygnn.GraphConv(-1, hidden_channels)
        self.act1 = nn.PReLU()
        self.conv2 = pygnn.GraphConv(-1, hidden_channels, add_self_loops=False)
        self.act2 = nn.PReLU()
        self.lin_out = pygnn.Linear(-1, out_channels)
        # self.probs_out = lambda x: x
        # if not regression_target:
        #     self.probs_out = nn.Softmax(dim=1)

    def forward(self, x, edge_index, batch=None):
        x = x.view(-1, 1)
        x = self.conv1(x, edge_index)
        x = self.act1(x)
        x = self.conv2(x, edge_index)
        x = self.act2(x)
        x = self.lin_out(x)
        # return self.probs_out(x)
        return x


model = HeteroHigherOrderGNN(32, 1, cs_hoeg_config["regression_task"])
model = pygnn.to_hetero(model, cs_hoeg_config["meta_data"])

# Print summary of data and model
cs_hoeg_config["verbose"] = True
if cs_hoeg_config["verbose"]:
    # print(model)
    with torch.no_grad():  # Initialize lazy modules, s.t. we can count its parameters.
        batch = next(iter(train_loader))
        batch.to(cs_hoeg_config["device"])
        model.to(cs_hoeg_config["device"])
        out = model(batch.x_dict, batch.edge_index_dict)
        print(f"Number of parameters: {utilities.torch_utils.count_parameters(model)}")

/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/nn/to_hetero_transformer.py:379: UserWarning: 'act1' will be duplicated, but its parameters cannot be reset. To suppress this warning, add a 'reset_parameters()' method to 'act1'
  warnings.warn(


/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/data/collate.py:160: UserWarning: An output with one or more elements was resized since it had shape [1], which does not match the required output shape [1, 21]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:17.)
  value = torch.cat(values, dim=cat_dim or 0, out=out)
/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/data/collate.py:160: UserWarning: An output with one or more elements was resized since it had shape [6], which does not match the required output shape [6, 21]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, i

Number of parameters: 808


In [ ]:
# MODEL TRAINING
print("Training started, progress available in Tensorboard")
torch.cuda.empty_cache()

timestamp = datetime.now().strftime("%Y%m%d_%Hh%Mm")
model_path_base = (
    f"{cs_hoeg_config['model_output_path']}/{str(model).split('(')[0]}_{timestamp}"
)

best_state_dict_path = hetero_training_utils.run_training_hetero(
    target_node_type=cs_hoeg_config["target_node_type"],
    num_epochs=cs_hoeg_config["EPOCHS"],
    model=model,
    train_loader=train_loader,
    validation_loader=val_loader,
    optimizer=O.Adam(model.parameters(), **cs_hoeg_config["optimizer_settings"]),
    loss_fn=cs_hoeg_config["loss_fn"],
    early_stopping_criterion=cs_hoeg_config["early_stopping"],
    model_path_base=model_path_base,
    device=cs_hoeg_config["device"],
    verbose=False,
    squeeze_required=cs_hoeg_config["squeeze_required"],
)

# Write experiment settings as JSON into model path (of the model we've just trained)
with open(os.path.join(model_path_base, "experiment_settings.json"), "w") as file_path:
    json.dump(evaluation_utils.get_json_serializable_dict(cs_hoeg_config), file_path)

Training started, progress available in Tensorboard


  0%|          | 0/5058 [00:00<?, ?it/s]

/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/data/collate.py:160: UserWarning: An output with one or more elements was resized since it had shape [2], which does not match the required output shape [2, 21]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:17.)
  value = torch.cat(values, dim=cat_dim or 0, out=out)
/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/data/collate.py:160: UserWarning: An output with one or more elements was resized since it had shape [11], which does not match the required output shape [11, 21]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it,

Early stopping after 9 epochs.


In [ ]:
# MODEL EVALUATION
model_path_base = f"{cs_hoeg_config['model_output_path']}/GraphModule_20230802_13h34m"
state_dict_path = f"{cs_hoeg_config['model_output_path']}/GraphModule_20230802_13h34m/state_dict_epoch2.pt"  # 0.5517 test mae | 15k params
# model_path_base = f"{cs_hoeg_config['model_output_path']}/GraphModule_20230802_16h49m"
# state_dict_path = f"{cs_hoeg_config['model_output_path']}/GraphModule_20230802_16h49m/state_dict_epoch4.pt"  # 0.5514 test mae | 808 params

# Get evaluation results
# evaluation_dict = hetero_evaluation_utils.evaluate_best_model(
evaluation_dict = evaluate_best_model(
    target_node_type=cs_hoeg_config["target_node_type"],
    model_state_dict_path=state_dict_path,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    model=model,
    metric=cs_hoeg_config["loss_fn"],
    device=cs_hoeg_config["device"],
    verbose=cs_hoeg_config["verbose"],
    squeeze_required=cs_hoeg_config["squeeze_required"],
)

# Store model results as JSON into model path
with open(os.path.join(model_path_base, "evaluation_report.json"), "w") as file_path:
    json.dump(evaluation_utils.get_json_serializable_dict(evaluation_dict), file_path)

  0%|          | 0/5058 [00:00<?, ?it/s]

/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/data/collate.py:160: UserWarning: An output with one or more elements was resized since it had shape [14], which does not match the required output shape [14, 21]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:17.)
  value = torch.cat(values, dim=cat_dim or 0, out=out)
/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/torch_geometric/data/collate.py:160: UserWarning: An output with one or more elements was resized since it had shape [6], which does not match the required output shape [6, 21]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it,

In [ ]:
# Print MAE results
print(model_path_base)
pprint(evaluation_dict)

models/CS/hoeg/GraphModule_20230802_13h34m
{'Test L1Loss()': tensor(0.5517),
 'Train L1Loss()': tensor(0.5572),
 'Val L1Loss()': tensor(0.5563)}
